# Reading OccurrenceCubeWERBirds data

In [16]:

import pandas as pd
import re
##file_path = 'https://github.com/FAIRiCUBE/data-requests/blob/main/encoding-examples/datacube_nl_farmland_birds_1.csv'
file_path =r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\datacube_nl_farmland_birds_1.csv"
df = pd.read_csv(file_path)
df.columns = ['year', 'gridnum_10m', 'species_id','B','C']

# Extraxt the EAST coordinate
def extract_east_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[1])  # Extract the second number
    else:
        return None

# Extraxt the NORTH coordinate
def extract_north_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[2])  # Extract the second number
    else:
        return None

# Apply the function to the text_column and create a new column
df['east_number'] = df['gridnum_10m'].apply(lambda x: extract_east_number(x))
df['north_number'] = df['gridnum_10m'].apply(lambda x: extract_north_number(x))

## get final coordinate: - for 10m reslolution:
df['east_coordinate_epsg3035_meter'] = df['east_number'] * 10
df['north_coordinate_epsg3035_meter'] = df['north_number'] * 10

df

,year,gridnum_10m,species_id,B,C,east_number,north_number,east_coordinate_epsg3035_meter,north_coordinate_epsg3035_meter
0,2018,10mE401855N329204,2480242,1,5000.0,401855,329204,4018550,3292040
1,2018,10mE401856N328589,2490266,1,5000.0,401856,328589,4018560,3285890
2,2018,10mE401868N329051,2494686,1,5000.0,401868,329051,4018680,3290510
3,2018,10mE401888N328568,2481819,1,5000.0,401888,328568,4018880,3285680
4,2018,10mE401894N329338,2480242,1,5000.0,401894,329338,4018940,3293380
...,...,...,...,...,...,...,...,...,...
3982,2018,10mE408294N330390,2481714,1,5000.0,408294,330390,4082940,3303900
3983,2018,10mE408295N330620,7788295,1,5000.0,408295,330620,4082950,3306200
3984,2018,10mE408308N330427,2480242,1,5000.0,408308,330427,4083080,3304270
3985,2018,10mE408311N330554,9809229,1,5000.0,408311,330554,4083110,3305540
